In [1]:
import splinepy
import gustaf as gus
import numpy as np
import vedo
from sdf_microstructure import (
    create_microstructure_from_experiment,
    export_mesh,
    tetrahedralize_surface,
)

from deep_sdf.analysis import linear_elasticity as le

vedo.settings.default_backend = "k3d"
tiling = [6,3,3]
tiling = [2,1,1]

In [2]:
faces, jacobian = create_microstructure_from_experiment("/home/michael.kofler/DeepSDF/experiments/round_cross_big_network", tiling=tiling, N_base=20)


In [3]:
volumes, surface_indices = tetrahedralize_surface(faces)

Delaunizing vertices...
Delaunay seconds:  0.033941
Creating surface mesh ...
Surface mesh seconds:  0.007675
Recovering boundaries...
Boundary recovery seconds:  0.138553
Removing exterior tetrahedra ...
Exterior tets removal seconds:  0.006431
Suppressing Steiner points ...
Steiner suppression seconds:  1.4e-05
Recovering Delaunayness...
Delaunay recovery seconds:  0.035381
Refining mesh...
Refinement seconds:  0.038857
Smoothing vertices...
Mesh smoothing seconds:  0.039888
Improving mesh...
Mesh improvement seconds:  0.010122
Jettisoning redundant points.

Writing nodes.
Writing elements.
Writing faces.
Writing edges.

Output seconds:  0.001239
Total running seconds:  0.318037

Statistics:

  Input points: 6794
  Input facets: 13608
  Input segments: 20412
  Input holes: 0
  Input regions: 0

  Mesh points: 7604
  Mesh tetrahedra: 25201
  Mesh faces: 57206
  Mesh faces on exterior boundary: 13608
  Mesh faces on input facets: 13608
  Mesh edges on input segments: 20412
  Steiner po

In [4]:
export_mesh(volumes, "test_mesh.mesh", show_mesh=False)
le_problem = le.LinearElasticityProblem()
le_problem.read_mesh("test_mesh.mesh")
le_problem.show_mesh()

Exporting mesh with 25201 elements, 7604 vertices, 896 boundaries with marker 1, 1064 boundaries with marker 2, and 11648 boundaries with marker 3.


K3DPlotterN(children=(Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680…

In [5]:
def dot_prod(A, B) -> np.ndarray:
    dot_ai_bi = (A * B).sum(axis=-1, keepdims=True)
    dot_bi_bi = (B * B).sum(axis=-1, keepdims=True)  # or square `norm`
    C = dot_ai_bi / dot_bi_bi * B
    return C

In [6]:
le_problem = le.LinearElasticityProblem()
le_problem.read_mesh("test_mesh.mesh")
# before solve, we should add a problem setup and set material properties
le_problem.set_up()
dVertices = None
dVertices = np.zeros((volumes.vertices.shape[0], volumes.vertices.shape[1], jacobian.shape[2]))
normals = gus.create.faces.vertex_normals(faces, angle_weighting=True, area_weighting=True)
dVertices_normal = np.zeros_like(jacobian)
for i in range(jacobian.shape[2]):
    dVertices_normal[:,:,i] = dot_prod(np.float64(jacobian[:,:,i]),normals.vertex_data["normals"])
    dVertices[surface_indices[:,0],:,i] = dVertices_normal[:,:,i]


(6794, 3)
(7604, 3)


K3DPlotterN(children=(Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680…

In [11]:
show_vol = []
for i in range(jacobian.shape[2]):
    volumes_cp = volumes.copy()
    volumes_cp.vertex_data["dVertices"] = -10*dVertices[:,:,i]
    volumes_cp.show_options["arrow_data"] = "dVertices"
    show_vol.append(volumes_cp)
gus.show(*show_vol)

K3DPlotterN(children=(Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680…

In [7]:
vol, der_vol = le_problem.compute_volume(dTheta=dVertices)
if der_vol is None:
    der_vol = 0
print(f"Vol: {vol:.5g}, dVol: {der_vol:.5g}")
le_problem.solve()
compliance, der_compliance = le_problem.compute_compliance(dTheta=dVertices)
if der_compliance is None:
    der_compliance = 0
print(f"Compliance: {compliance:.5g}, dCompliance: {der_compliance:.5g}")
le_problem.show_solution(output=["u_vec","strain_energy_density"])
expected_vol = vol+der_vol
expected_compl = compliance + der_compliance

TypeError: unsupported format string passed to numpy.ndarray.__format__

In [17]:
le_problem = le.LinearElasticityProblem()
volumes_stretched = volumes.copy()
volumes_stretched.vertices = volumes.vertices + dVertices
export_mesh(volumes_stretched, "test_mesh_stretched.mesh")
le_problem.read_mesh("test_mesh_stretched.mesh")
# before solve, we should add a problem setup and set material properties
le_problem.set_up(ref_levels=0)
vol, _ = le_problem.compute_volume()
print(f"Volume of deformed mesh {vol:.5g} ({expected_vol:.5g} expected)")
le_problem.solve()
compliance, der_compliance = le_problem.compute_compliance()
print(f"Compliance of deformed mesh: {compliance:.5g} ({expected_compl:.5g} expected)")
le_problem.show_solution(output=["u_vec","strain_energy_density"])

Exporting mesh with 414497 elements, 113665 vertices, 7080 boundaries with marker 1, 7568 boundaries with marker 2, and 166376 boundaries with marker 3.
Elements with wrong orientation: 1698 / 414497 (fixed)
Volume of deformed mesh 0.85541 (0.8544 expected)
Finished Solution. Max deflection: 0.0015224935999991402
Saving results to linear_elasticity
Compliance of deformed mesh: 17.68 (19.256 expected)


K3DPlotterN(children=(Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680…

In [18]:
volumes_stretched.vertex_data["dVertices"] = 10*dVertices
# mag_dvert = np.linalg.norm(dVertices, axis=1)
# volumes.vertex_data["directions_magnitude"] = mag_dvert
# volumes.show_options["data"] = "directions_magnitue"
volumes_stretched.show_options["arrow_data"] = "dVertices"
gus.show(volumes_stretched)

K3DPlotterN(children=(Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680…